# 49-异步网络编程

## 📚 用途说明

**学习目标**：
- 掌握异步HTTP客户端和服务器编程技术
- 学会使用aiohttp进行高效的异步网络请求
- 理解异步WebSocket连接和实时通信机制
- 能够构建高性能的异步网络应用系统

**前置要求**：
- 已完成异步I/O操作学习
- 熟练掌握async/await语法和协程机制
- 了解基本的HTTP协议和网络编程概念
- 理解异步编程的网络性能优势

**与LangChain关联**：
- 异步网络请求优化LangChain的API调用性能
- WebSocket支持LangChain的实时流式响应
- 并发HTTP请求提升LangChain的数据获取效率
- 异步网络编程增强LangChain的分布式能力

---

## 🔢 知识点覆盖

### 6.4 异步网络编程 [⭐⭐进阶]
**知识点说明**：异步网络编程是构建高性能网络应用的核心技术，对于LangChain的分布式和实时处理至关重要。

**学习要求**：
- 掌握异步HTTP客户端和服务器编程技术
- 学会使用aiohttp进行高效的异步网络请求
- 理解异步WebSocket连接和实时通信机制
- 能够构建高性能的异步网络应用系统

**案例要求**：
- 实现完整的异步HTTP客户端和服务器
- 进行异步网络性能对比分析
- 应用异步网络技术解决实际问题
- 验证点：能独立优化异步网络性能

In [ ]:
print("🌐 异步网络编程:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import json
import random
import ssl
from typing import List, Dict, Any, Optional, AsyncGenerator
from dataclasses import dataclass, asdict
import aiohttp
import aiohttp.web
from aiohttp import ClientSession, ClientTimeout, TCPConnector
import nest_asyncio

# 允许在Jupyter中运行异步代码
nest_asyncio.apply()

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ aiohttp版本: {aiohttp.__version__}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 异步HTTP客户端基础
print(f"📝 1. 异步HTTP客户端基础:")

# 1.1 基本HTTP请求
print(f"\n   🔍 1.1 基本HTTP请求:")

async def demonstrate_basic_http_requests():
    """演示基本HTTP请求"""
    print(f"   🔸 基本HTTP请求演示:")
    
    # 使用httpbin.org进行测试
    test_urls = [
        "https://httpbin.org/get",
        "https://httpbin.org/delay/1",
        "https://httpbin.org/json"
    ]
    
    async with ClientSession() as session:
        for i, url in enumerate(test_urls, 1):
            print(f"\n      请求 {i}: {url}")
            
            try:
                start_time = time.time()
                
                async with session.get(url) as response:
                    status = response.status
                    content_type = response.headers.get('content-type', 'unknown')
                    
                    if content_type == 'application/json':
                        data = await response.json()
                        print(f"        状态码: {status}")
                        print(f"        响应类型: {content_type}")
                        print(f"        数据键: {list(data.keys())}")
                    else:
                        text = await response.text()
                        print(f"        状态码: {status}")
                        print(f"        响应类型: {content_type}")
                        print(f"        内容长度: {len(text)} 字符")
                    
                    end_time = time.time()
                    print(f"        请求耗时: {end_time - start_time:.3f} 秒")
                    
            except Exception as e:
                print(f"        请求失败: {e}")
    
    return "基本HTTP请求演示完成"

# 运行基本HTTP请求演示
basic_http_result = await demonstrate_basic_http_requests()
print(f"   {basic_http_result}")

# 1.2 并发HTTP请求
print(f"\n   🚀 1.2 并发HTTP请求:")

async def fetch_url(session: ClientSession, url: str, request_id: int) -> Dict[str, Any]:
    """获取单个URL"""
    try:
        start_time = time.time()
        
        async with session.get(url) as response:
            data = await response.json()
            end_time = time.time()
            
            return {
                'request_id': request_id,
                'url': url,
                'status': response.status,
                'response_time': end_time - start_time,
                'origin': data.get('origin', 'unknown')
            }
    except Exception as e:
        return {
            'request_id': request_id,
            'url': url,
            'error': str(e),
            'response_time': 0
        }

async def demonstrate_concurrent_requests():
    """演示并发HTTP请求"""
    print(f"   🔸 并发HTTP请求演示:")
    
    # 创建多个测试URL
    base_url = "https://httpbin.org"
    endpoints = [
        "/get",
        "/delay/1",
        "/uuid",
        "/ip",
        "/user-agent"
    ]
    
    urls = [base_url + endpoint for endpoint in endpoints]
    
    # 配置连接器
    connector = TCPConnector(limit=100, limit_per_host=20)
    timeout = ClientTimeout(total=10, connect=3)
    
    async with ClientSession(connector=connector, timeout=timeout) as session:
        print(f"\n      并发请求 {len(urls)} 个URL...")
        
        # 创建并发任务
        tasks = [
            fetch_url(session, url, i+1) 
            for i, url in enumerate(urls)
        ]
        
        start_time = time.time()
        
        # 执行并发请求
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        end_time = time.time()
        total_time = end_time - start_time
        
        print(f"\n      并发请求完成:")
        print(f"        总耗时: {total_time:.3f} 秒")
        print(f"        成功请求: {sum(1 for r in results if 'error' not in r)}")
        print(f"        失败请求: {sum(1 for r in results if 'error' in r)}")
        
        # 显示详细结果
        print(f"\n      详细结果:")
        for result in results:
            if isinstance(result, dict) and 'error' not in result:
                print(f"        请求{result['request_id']}: {result['url']}")
                print(f"          状态: {result['status']}, 耗时: {result['response_time']:.3f}s")
            elif isinstance(result, dict):
                print(f"        请求{result['request_id']}: {result['url']}")
                print(f"          错误: {result['error']}")
        
        # 计算性能指标
        successful_results = [r for r in results if isinstance(r, dict) and 'error' not in r]
        if successful_results:
            avg_response_time = sum(r['response_time'] for r in successful_results) / len(successful_results)
            max_response_time = max(r['response_time'] for r in successful_results)
            min_response_time = min(r['response_time'] for r in successful_results)
            
            print(f"\n      性能指标:")
            print(f"        平均响应时间: {avg_response_time:.3f} 秒")
            print(f"        最大响应时间: {max_response_time:.3f} 秒")
            print(f"        最小响应时间: {min_response_time:.3f} 秒")
            print(f"        并发效率: {len(successful_results)/total_time:.1f} 请求/秒")
    
    return "并发HTTP请求演示完成"

# 运行并发请求演示
concurrent_result = await demonstrate_concurrent_requests()
print(f"   {concurrent_result}")

# 1.3 高级HTTP客户端配置
print(f"\n   ⚙️ 1.3 高级HTTP客户端配置:")

@dataclass
class HttpClientConfig:
    """HTTP客户端配置"""
    timeout_total: int = 30
    timeout_connect: int = 5
    limit_connections: int = 100
    limit_per_host: int = 20
    retry_attempts: int = 3
    retry_delay: float = 1.0

class AsyncHttpClient:
    """高级异步HTTP客户端"""
    
    def __init__(self, config: HttpClientConfig):
        self.config = config
        self.session = None
    
    async def __aenter__(self):
        """异步上下文管理器入口"""
        connector = TCPConnector(
            limit=self.config.limit_connections,
            limit_per_host=self.config.limit_per_host
        )
        timeout = ClientTimeout(
            total=self.config.timeout_total,
            connect=self.config.timeout_connect
        )
        
        self.session = ClientSession(
            connector=connector,
            timeout=timeout
        )
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """异步上下文管理器出口"""
        if self.session:
            await self.session.close()
    
    async def get_with_retry(self, url: str, **kwargs) -> Dict[str, Any]:
        """带重试的GET请求"""
        last_exception = None
        
        for attempt in range(self.config.retry_attempts):
            try:
                async with self.session.get(url, **kwargs) as response:
                    if response.status == 200:
                        return await response.json()
                    else:
                        raise aiohttp.ClientResponseError(
                            request_info=response.request_info,
                            history=response.history,
                            status=response.status,
                            message=f"HTTP {response.status}"
                        )
            except Exception as e:
                last_exception = e
                if attempt < self.config.retry_attempts - 1:
                    await asyncio.sleep(self.config.retry_delay * (2 ** attempt))
                    print(f"        重试第{attempt + 1}次: {url}")
        
        raise last_exception

async def demonstrate_advanced_client():
    """演示高级HTTP客户端"""
    print(f"   🔸 高级HTTP客户端演示:")
    
    # 创建客户端配置
    config = HttpClientConfig(
        timeout_total=10,
        timeout_connect=3,
        limit_connections=50,
        limit_per_host=10,
        retry_attempts=2,
        retry_delay=0.5
    )
    
    print(f"      客户端配置:")
    print(f"        超时设置: {config.timeout_total}s总超时, {config.timeout_connect}s连接超时")
    print(f"        连接限制: {config.limit_connections}总连接, {config.limit_per_host}每主机")
    print(f"        重试策略: {config.retry_attempts}次重试, {config.retry_delay}s延迟")
    
    # 测试URL列表
    test_urls = [
        "https://httpbin.org/get",
        "https://httpbin.org/status/500",  # 会触发重试
        "https://httpbin.org/delay/2"
    ]
    
    async with AsyncHttpClient(config) as client:
        for i, url in enumerate(test_urls, 1):
            print(f"\n      测试 {i}: {url}")
            
            try:
                start_time = time.time()
                result = await client.get_with_retry(url)
                end_time = time.time()
                
                print(f"        请求成功: {list(result.keys())}")
                print(f"        总耗时: {end_time - start_time:.3f} 秒")
                
            except Exception as e:
                print(f"        请求失败: {e}")
    
    return "高级HTTP客户端演示完成"

# 运行高级客户端演示
advanced_client_result = await demonstrate_advanced_client()
print(f"   {advanced_client_result}")

print(f"\n✅ 异步HTTP客户端基础完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步HTTP客户端和服务器编程技术")
print(f"   ✓ 学会使用aiohttp进行高效的异步网络请求")
print(f"   ✓ 理解并发HTTP请求和性能优化")
print(f"   ✓ 掌握高级HTTP客户端配置和重试机制")

### 异步WebSocket编程 [⭐⭐进阶]
**知识点说明**：异步WebSocket编程是实现实时双向通信的关键技术，对于LangChain的流式响应和实时交互非常重要。

**学习要求**：
- 掌握异步WebSocket客户端和服务器技术
- 理解WebSocket连接管理和消息处理
- 学会实时数据流和双向通信实现
- 能够构建高效的WebSocket应用系统

**案例要求**：
- 实现完整的异步WebSocket客户端
- 进行WebSocket性能和连接管理分析
- 应用WebSocket技术解决实际问题
- 验证点：能独立实现WebSocket实时通信

In [ ]:
print("\n🔌 异步WebSocket编程:")
print("=" * 50)

# 1. WebSocket客户端基础
print(f"📝 1. WebSocket客户端基础:")

# 1.1 WebSocket连接管理
print(f"\n   🔍 1.1 WebSocket连接管理:")

class WebSocketClient:
    """WebSocket客户端"""
    
    def __init__(self, url: str):
        self.url = url
        self.session = None
        self.websocket = None
        self.connected = False
    
    async def connect(self):
        """连接WebSocket"""
        try:
            self.session = ClientSession()
            self.websocket = await self.session.ws_connect(self.url)
            self.connected = True
            print(f"        WebSocket连接成功: {self.url}")
            return True
        except Exception as e:
            print(f"        WebSocket连接失败: {e}")
            return False
    
    async def send_message(self, message: str):
        """发送消息"""
        if self.connected and self.websocket:
            await self.websocket.send_str(message)
            print(f"        发送消息: {message}")
        else:
            print(f"        WebSocket未连接")
    
    async def receive_message(self, timeout: float = 5.0) -> Optional[str]:
        """接收消息"""
        if self.connected and self.websocket:
            try:
                message = await asyncio.wait_for(
                    self.websocket.receive_str(), 
                    timeout=timeout
                )
                print(f"        接收消息: {message}")
                return message
            except asyncio.TimeoutError:
                print(f"        接收消息超时")
                return None
            except Exception as e:
                print(f"        接收消息失败: {e}")
                return None
        else:
            print(f"        WebSocket未连接")
            return None
    
    async def close(self):
        """关闭连接"""
        if self.websocket:
            await self.websocket.close()
            self.connected = False
            print(f"        WebSocket连接已关闭")
        if self.session:
            await self.session.close()

async def demonstrate_websocket_client():
    """演示WebSocket客户端"""
    print(f"   🔸 WebSocket客户端演示:")
    
    # 使用公共WebSocket测试服务
    # 注意：实际使用时可能需要替换为可用的WebSocket服务
    test_urls = [
        "wss://echo.websocket.org",  # WebSocket回显服务
        "wss://ws.postman-echo.com/raw"  # Postman WebSocket测试
    ]
    
    for url in test_urls:
        print(f"\n      测试WebSocket服务: {url}")
        
        client = WebSocketClient(url)
        
        try:
            # 连接WebSocket
            if await client.connect():
                # 发送测试消息
                test_messages = [
                    "Hello WebSocket!",
                    "测试中文消息",
                    json.dumps({"type": "test", "data": "example"})
                ]
                
                for message in test_messages:
                    await client.send_message(message)
                    response = await client.receive_message(timeout=3.0)
                    
                    if response:
                        print(f"          响应匹配: {message == response}")
                    
                    await asyncio.sleep(0.5)  # 间隔
                
                # 关闭连接
                await client.close()
            
        except Exception as e:
            print(f"        WebSocket测试失败: {e}")
            await client.close()
    
    return "WebSocket客户端演示完成"

# 运行WebSocket客户端演示
# 注意：由于网络限制，这里使用模拟演示
print(f"      注意: 由于Jupyter环境限制，使用模拟WebSocket演示")

async def simulate_websocket_demo():
    """模拟WebSocket演示"""
    print(f"   🔸 模拟WebSocket客户端演示:")
    
    # 模拟WebSocket连接过程
    print(f"\n      模拟连接WebSocket服务器...")
    await asyncio.sleep(0.5)
    print(f"        WebSocket连接成功: wss://echo.example.com")
    
    # 模拟消息交换
    test_messages = [
        "Hello WebSocket!",
        "测试中文消息",
        json.dumps({"type": "test", "data": "example"})
    ]
    
    for message in test_messages:
        print(f"        发送消息: {message}")
        await asyncio.sleep(0.2)
        print(f"        接收消息: {message} (回显)")
        print(f"          响应匹配: True")
        await asyncio.sleep(0.3)
    
    print(f"\n        WebSocket连接已关闭")
    
    return "模拟WebSocket客户端演示完成"

# 运行模拟WebSocket演示
websocket_result = await simulate_websocket_demo()
print(f"   {websocket_result}")

# 1.2 实时数据流处理
print(f"\n   🌊 1.2 实时数据流处理:")

class StreamingWebSocketClient:
    """流式WebSocket客户端"""
    
    def __init__(self, url: str):
        self.url = url
        self.is_streaming = False
        self.message_count = 0
    
    async def start_streaming(self, duration: float = 10.0):
        """开始流式接收数据"""
        print(f"      开始流式接收数据，持续时间: {duration} 秒")
        
        self.is_streaming = True
        self.message_count = 0
        
        start_time = time.time()
        
        # 模拟流式数据接收
        while self.is_streaming and (time.time() - start_time) < duration:
            # 模拟接收消息
            await asyncio.sleep(0.5)
            
            self.message_count += 1
            message = {
                "id": self.message_count,
                "timestamp": time.time(),
                "data": f"流式数据{self.message_count}",
                "random": random.randint(1, 100)
            }
            
            print(f"        接收流式消息 #{self.message_count}: {json.dumps(message, ensure_ascii=False)}")
        
        end_time = time.time()
        actual_duration = end_time - start_time
        
        print(f"\n      流式接收完成:")
        print(f"        实际持续时间: {actual_duration:.3f} 秒")
        print(f"        接收消息数量: {self.message_count}")
        print(f"        平均消息速率: {self.message_count/actual_duration:.1f} 消息/秒")
        
        self.is_streaming = False
    
    async def stop_streaming(self):
        """停止流式接收"""
        self.is_streaming = False
        print(f"      停止流式接收")
    
    async def send_ping(self):
        """发送心跳包"""
        ping_message = {
            "type": "ping",
            "timestamp": time.time()
        }
        print(f"        发送心跳: {json.dumps(ping_message)}")

async def demonstrate_streaming_websocket():
    """演示流式WebSocket"""
    print(f"   🔸 流式WebSocket演示:")
    
    client = StreamingWebSocketClient("wss://stream.example.com")
    
    try:
        # 启动流式接收
        streaming_task = asyncio.create_task(client.start_streaming(8.0))
        
        # 定期发送心跳
        ping_task = asyncio.create_task(
            periodic_ping(client, interval=2.0, count=4)
        )
        
        # 等待任务完成
        await asyncio.gather(streaming_task, ping_task)
        
    except Exception as e:
        print(f"        流式演示失败: {e}")
        await client.stop_streaming()
    
    return "流式WebSocket演示完成"

async def periodic_ping(client: StreamingWebSocketClient, interval: float, count: int):
    """定期发送心跳"""
    for i in range(count):
        await asyncio.sleep(interval)
        if client.is_streaming:
            await client.send_ping()
        else:
            break

# 运行流式WebSocket演示
streaming_result = await demonstrate_streaming_websocket()
print(f"   {streaming_result}")

# 1.3 WebSocket连接池管理
print(f"\n   🏊 1.3 WebSocket连接池管理:")

class WebSocketPool:
    """WebSocket连接池"""
    
    def __init__(self, max_connections: int = 5):
        self.max_connections = max_connections
        self.connections = {}
        self.available_connections = asyncio.Queue(maxsize=max_connections)
        self.connection_count = 0
    
    async def get_connection(self, url: str) -> str:
        """获取连接"""
        connection_id = f"conn_{self.connection_count}"
        
        if connection_id not in self.connections:
            if self.connection_count >= self.max_connections:
                print(f"        连接池已满，等待可用连接...")
                await self.available_connections.get()
            
            # 创建新连接
            self.connections[connection_id] = {
                'url': url,
                'created_at': time.time(),
                'message_count': 0
            }
            
            self.connection_count += 1
            print(f"        创建新连接: {connection_id} -> {url}")
        
        return connection_id
    
    async def release_connection(self, connection_id: str):
        """释放连接"""
        if connection_id in self.connections:
            await self.available_connections.put(connection_id)
            print(f"        释放连接: {connection_id}")
    
    async def send_message(self, connection_id: str, message: str):
        """发送消息"""
        if connection_id in self.connections:
            self.connections[connection_id]['message_count'] += 1
            print(f"        {connection_id} 发送消息 #{self.connections[connection_id]['message_count']}: {message}")
    
    def get_pool_stats(self) -> Dict[str, Any]:
        """获取连接池统计"""
        total_messages = sum(
            conn['message_count'] for conn in self.connections.values()
        )
        
        return {
            'total_connections': len(self.connections),
            'max_connections': self.max_connections,
            'total_messages': total_messages,
            'available_slots': self.available_connections.qsize()
        }

async def demonstrate_websocket_pool():
    """演示WebSocket连接池"""
    print(f"   🔸 WebSocket连接池演示:")
    
    pool = WebSocketPool(max_connections=3)
    
    # 模拟多个客户端使用连接池
    async def client_task(client_id: int, message_count: int):
        url = f"wss://server{client_id % 3 + 1}.example.com"
        
        connection_id = await pool.get_connection(url)
        
        for i in range(message_count):
            message = f"客户端{client_id}消息{i+1}"
            await pool.send_message(connection_id, message)
            await asyncio.sleep(0.1)
        
        await pool.release_connection(connection_id)
        
        print(f"        客户端{client_id}完成")
    
    # 创建多个客户端任务
    client_tasks = [
        client_task(i, random.randint(3, 7))
        for i in range(5)
    ]
    
    print(f"\n      启动5个客户端任务，最大连接数3...")
    await asyncio.gather(*client_tasks)
    
    # 显示连接池统计
    stats = pool.get_pool_stats()
    print(f"\n      连接池统计:")
    for key, value in stats.items():
        print(f"        {key}: {value}")
    
    return "WebSocket连接池演示完成"

# 运行WebSocket连接池演示
pool_result = await demonstrate_websocket_pool()
print(f"   {pool_result}")

print(f"\n✅ 异步WebSocket编程完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步WebSocket客户端和服务器技术")
print(f"   ✓ 理解WebSocket连接管理和消息处理")
print(f"   ✓ 学会实时数据流和双向通信实现")
print(f"   ✓ 能够构建高效的WebSocket应用系统")

### 异步网络性能优化 [⭐⭐进阶]
**知识点说明**：异步网络性能优化是构建高性能网络应用的关键，对于LangChain的大规模并发处理至关重要。

**学习要求**：
- 掌握异步网络性能分析和优化技术
- 理解连接池、缓存和负载均衡机制
- 学会网络监控和故障处理策略
- 能够实现高效的异步网络系统

**案例要求**：
- 实现完整的异步网络性能优化系统
- 进行网络性能对比和分析
- 应用优化技术解决实际问题
- 验证点：能独立优化异步网络性能

In [ ]:
print("\n⚡ 异步网络性能优化:")
print("=" * 50)

# 1. 性能监控和分析
print(f"📝 1. 性能监控和分析:")

# 1.1 网络性能指标收集
print(f"\n   🔍 1.1 网络性能指标收集:")

@dataclass
class NetworkMetrics:
    """网络性能指标"""
    request_count: int = 0
    success_count: int = 0
    error_count: int = 0
    total_response_time: float = 0.0
    min_response_time: float = float('inf')
    max_response_time: float = 0.0
    bytes_received: int = 0
    bytes_sent: int = 0
    start_time: float = 0.0
    end_time: float = 0.0
    
    @property
    def success_rate(self) -> float:
        """成功率"""
        if self.request_count == 0:
            return 0.0
        return self.success_count / self.request_count * 100
    
    @property
    def average_response_time(self) -> float:
        """平均响应时间"""
        if self.success_count == 0:
            return 0.0
        return self.total_response_time / self.success_count
    
    @property
    def requests_per_second(self) -> float:
        """每秒请求数"""
        duration = self.end_time - self.start_time
        if duration == 0:
            return 0.0
        return self.request_count / duration
    
    def update(self, response_time: float, success: bool, bytes_received: int = 0, bytes_sent: int = 0):
        """更新指标"""
        self.request_count += 1
        
        if success:
            self.success_count += 1
            self.total_response_time += response_time
            self.min_response_time = min(self.min_response_time, response_time)
            self.max_response_time = max(self.max_response_time, response_time)
        else:
            self.error_count += 1
        
        self.bytes_received += bytes_received
        self.bytes_sent += bytes_sent

class PerformanceMonitor:
    """性能监控器"""
    
    def __init__(self):
        self.metrics = NetworkMetrics()
        self.real_time_metrics = []
    
    def start_monitoring(self):
        """开始监控"""
        self.metrics.start_time = time.time()
        print(f"        开始性能监控")
    
    def stop_monitoring(self):
        """停止监控"""
        self.metrics.end_time = time.time()
        print(f"        停止性能监控")
    
    def record_request(self, response_time: float, success: bool, bytes_received: int = 0, bytes_sent: int = 0):
        """记录请求"""
        self.metrics.update(response_time, success, bytes_received, bytes_sent)
        
        # 记录实时指标
        self.real_time_metrics.append({
            'timestamp': time.time(),
            'response_time': response_time,
            'success': success
        })
    
    def get_summary(self) -> Dict[str, Any]:
        """获取性能摘要"""
        return {
            '总请求数': self.metrics.request_count,
            '成功请求数': self.metrics.success_count,
            '失败请求数': self.metrics.error_count,
            '成功率': f"{self.metrics.success_rate:.1f}%",
            '平均响应时间': f"{self.metrics.average_response_time:.3f}s",
            '最小响应时间': f"{self.metrics.min_response_time:.3f}s",
            '最大响应时间': f"{self.metrics.max_response_time:.3f}s",
            '每秒请求数': f"{self.metrics.requests_per_second:.1f}",
            '接收字节': self.metrics.bytes_received,
            '发送字节': self.metrics.bytes_sent,
            '总耗时': f"{self.metrics.end_time - self.metrics.start_time:.3f}s"
        }

async def demonstrate_performance_monitoring():
    """演示性能监控"""
    print(f"   🔸 网络性能监控演示:")
    
    monitor = PerformanceMonitor()
    monitor.start_monitoring()
    
    # 模拟网络请求
    async def simulated_request(request_id: int):
        # 模拟网络延迟
        await asyncio.sleep(random.uniform(0.1, 0.5))
        
        # 模拟请求结果
        success = random.random() > 0.1  # 90%成功率
        response_time = random.uniform(0.1, 0.8)
        bytes_received = random.randint(100, 1000)
        bytes_sent = random.randint(50, 200)
        
        monitor.record_request(response_time, success, bytes_received, bytes_sent)
        
        return success
    
    # 执行并发请求
    print(f"\n      执行20个并发网络请求...")
    tasks = [simulated_request(i) for i in range(20)]
    results = await asyncio.gather(*tasks)
    
    monitor.stop_monitoring()
    
    # 显示性能摘要
    summary = monitor.get_summary()
    print(f"\n      性能监控摘要:")
    for key, value in summary.items():
        print(f"        {key}: {value}")
    
    return "网络性能监控演示完成"

# 运行性能监控演示
monitoring_result = await demonstrate_performance_monitoring()
print(f"   {monitoring_result}")

# 1.2 连接池优化
print(f"\n   🏊 1.2 连接池优化:")

class OptimizedConnectionPool:
    """优化的连接池"""
    
    def __init__(self, max_connections: int = 10, max_per_host: int = 5):
        self.max_connections = max_connections
        self.max_per_host = max_per_host
        self.session = None
        self.connection_stats = {}
    
    async def __aenter__(self):
        """创建优化的连接器"""
        connector = TCPConnector(
            limit=self.max_connections,
            limit_per_host=self.max_per_host,
            ttl_dns_cache=300,  # DNS缓存5分钟
            use_dns_cache=True,
            keepalive_timeout=30,  # 保持连接30秒
            enable_cleanup_closed=True
        )
        
        timeout = ClientTimeout(
            total=30,
            connect=5,
            sock_read=10
        )
        
        self.session = ClientSession(
            connector=connector,
            timeout=timeout
        )
        
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """清理连接池"""
        if self.session:
            await self.session.close()
    
    async def fetch_with_stats(self, url: str, host: str = None) -> Dict[str, Any]:
        """带统计的请求"""
        if host is None:
            host = url.split('/')[2]
        
        start_time = time.time()
        
        try:
            async with self.session.get(url) as response:
                data = await response.json()
                end_time = time.time()
                
                # 更新统计
                if host not in self.connection_stats:
                    self.connection_stats[host] = {
                        'requests': 0,
                        'successes': 0,
                        'total_time': 0.0
                    }
                
                stats = self.connection_stats[host]
                stats['requests'] += 1
                stats['successes'] += 1
                stats['total_time'] += end_time - start_time
                
                return {
                    'url': url,
                    'host': host,
                    'status': response.status,
                    'response_time': end_time - start_time,
                    'success': True
                }
        
        except Exception as e:
            end_time = time.time()
            
            # 更新失败统计
            if host not in self.connection_stats:
                self.connection_stats[host] = {
                    'requests': 0,
                    'successes': 0,
                    'total_time': 0.0
                }
            
            stats = self.connection_stats[host]
            stats['requests'] += 1
            
            return {
                'url': url,
                'host': host,
                'error': str(e),
                'response_time': end_time - start_time,
                'success': False
            }
    
    def get_connection_stats(self) -> Dict[str, Any]:
        """获取连接统计"""
        stats_summary = {}
        
        for host, stats in self.connection_stats.items():
            if stats['successes'] > 0:
                avg_time = stats['total_time'] / stats['successes']
            else:
                avg_time = 0.0
            
            success_rate = (stats['successes'] / stats['requests'] * 100) if stats['requests'] > 0 else 0
            
            stats_summary[host] = {
                'requests': stats['requests'],
                'successes': stats['successes'],
                'success_rate': f"{success_rate:.1f}%",
                'avg_response_time': f"{avg_time:.3f}s"
            }
        
        return stats_summary

async def demonstrate_connection_pool_optimization():
    """演示连接池优化"""
    print(f"   🔸 连接池优化演示:")
    
    # 测试不同主机的URL
    test_urls = [
        ("https://httpbin.org/get", "httpbin.org"),
        ("https://httpbin.org/delay/1", "httpbin.org"),
        ("https://httpbin.org/json", "httpbin.org"),
        ("https://jsonplaceholder.typicode.com/posts/1", "jsonplaceholder.typicode.com"),
        ("https://jsonplaceholder.typicode.com/posts/2", "jsonplaceholder.typicode.com"),
    ]
    
    async with OptimizedConnectionPool(max_connections=8, max_per_host=4) as pool:
        print(f"\n      连接池配置: 最大8连接, 每主机4连接")
        print(f"\n      执行并发请求...")
        
        # 创建并发任务
        tasks = [
            pool.fetch_with_stats(url, host)
            for url, host in test_urls
        ]
        
        start_time = time.time()
        results = await asyncio.gather(*tasks)
        end_time = time.time()
        
        print(f"\n      请求完成，总耗时: {end_time - start_time:.3f} 秒")
        
        # 显示结果
        successful_results = [r for r in results if r['success']]
        print(f"\n      请求结果:")
        print(f"        成功请求: {len(successful_results)}")
        print(f"        失败请求: {len(results) - len(successful_results)}")
        
        # 显示连接统计
        connection_stats = pool.get_connection_stats()
        print(f"\n      连接统计:")
        for host, stats in connection_stats.items():
            print(f"        主机 {host}:")
            for key, value in stats.items():
                print(f"          {key}: {value}")
    
    return "连接池优化演示完成"

# 运行连接池优化演示
pool_optimization_result = await demonstrate_connection_pool_optimization()
print(f"   {pool_optimization_result}")

# 1.3 负载均衡和故障处理
print(f"\n   ⚖️ 1.3 负载均衡和故障处理:")

class LoadBalancer:
    """负载均衡器"""
    
    def __init__(self, servers: List[str]):
        self.servers = servers
        self.server_stats = {server: {'requests': 0, 'failures': 0, 'last_used': 0} for server in servers}
        self.current_index = 0
    
    def get_server_round_robin(self) -> str:
        """轮询选择服务器"""
        server = self.servers[self.current_index]
        self.current_index = (self.current_index + 1) % len(self.servers)
        return server
    
    def get_server_least_connections(self) -> str:
        """最少连接选择服务器"""
        return min(self.servers, key=lambda s: self.server_stats[s]['requests'])
    
    def get_server_weighted(self, weights: List[float]) -> str:
        """加权选择服务器"""
        if len(weights) != len(self.servers):
            return self.get_server_round_robin()
        
        total_weight = sum(weights)
        rand = random.random() * total_weight
        
        current_weight = 0
        for server, weight in zip(self.servers, weights):
            current_weight += weight
            if rand <= current_weight:
                return server
        
        return self.servers[-1]
    
    def record_request(self, server: str, success: bool):
        """记录请求结果"""
        if server in self.server_stats:
            self.server_stats[server]['requests'] += 1
            self.server_stats[server]['last_used'] = time.time()
            
            if not success:
                self.server_stats[server]['failures'] += 1
    
    def get_healthy_servers(self, max_failure_rate: float = 0.5) -> List[str]:
        """获取健康服务器列表"""
        healthy = []
        
        for server, stats in self.server_stats.items():
            if stats['requests'] == 0:
                failure_rate = 0.0
            else:
                failure_rate = stats['failures'] / stats['requests']
            
            if failure_rate <= max_failure_rate:
                healthy.append(server)
        
        return healthy if healthy else self.servers  # 如果所有都不健康，返回所有

class FaultTolerantClient:
    """容错客户端"""
    
    def __init__(self, load_balancer: LoadBalancer, retry_attempts: int = 3):
        self.load_balancer = load_balancer
        self.retry_attempts = retry_attempts
    
    async def request_with_fallback(self, endpoint: str, method: str = 'GET') -> Dict[str, Any]:
        """带故障转移的请求"""
        healthy_servers = self.load_balancer.get_healthy_servers()
        
        for attempt in range(self.retry_attempts):
            for server in healthy_servers:
                url = f"{server}{endpoint}"
                
                try:
                    start_time = time.time()
                    
                    async with ClientSession() as session:
                        async with session.get(url) as response:
                            if response.status == 200:
                                data = await response.json()
                                end_time = time.time()
                                
                                self.load_balancer.record_request(server, True)
                                
                                return {
                                    'server': server,
                                    'url': url,
                                    'status': response.status,
                                    'response_time': end_time - start_time,
                                    'success': True,
                                    'attempt': attempt + 1
                                }
                            else:
                                raise aiohttp.ClientResponseError(
                                    request_info=None,
                                    history=None,
                                    status=response.status,
                                    message=f"HTTP {response.status}"
                                )
                
                except Exception as e:
                    self.load_balancer.record_request(server, False)
                    print(f"        服务器 {server} 请求失败: {e}")
                    continue
        
        return {
            'server': 'none',
            'url': endpoint,
            'error': 'All servers failed',
            'success': False,
            'attempts': self.retry_attempts
        }

async def demonstrate_load_balancing():
    """演示负载均衡"""
    print(f"   🔸 负载均衡和故障处理演示:")
    
    # 模拟服务器列表
    servers = [
        "https://httpbin.org",
        "https://jsonplaceholder.typicode.com",
        "https://api.github.com"
    ]
    
    load_balancer = LoadBalancer(servers)
    fault_tolerant_client = FaultTolerantClient(load_balancer)
    
    # 测试不同的负载均衡策略
    strategies = [
        ("轮询", lambda: load_balancer.get_server_round_robin()),
        ("最少连接", lambda: load_balancer.get_server_least_connections()),
        ("加权", lambda: load_balancer.get_server_weighted([0.5, 0.3, 0.2]))
    ]
    
    for strategy_name, strategy_func in strategies:
        print(f"\n      测试{strategy_name}策略:")
        
        # 重置统计
        load_balancer.server_stats = {server: {'requests': 0, 'failures': 0, 'last_used': 0} for server in servers}
        
        # 模拟请求分配
        for i in range(9):
            server = strategy_func()
            print(f"        请求{i+1} -> {server}")
            load_balancer.record_request(server, True)
        
        # 显示分配结果
        print(f"        分配结果:")
        for server, stats in load_balancer.server_stats.items():
            print(f"          {server}: {stats['requests']} 请求")
    
    # 测试故障转移
    print(f"\n      测试故障转移:")
    
    # 故意设置一些服务器为不健康
    load_balancer.record_request(servers[0], False)  # 标记为失败
    load_balancer.record_request(servers[0], False)
    
    healthy_servers = load_balancer.get_healthy_servers(max_failure_rate=0.3)
    print(f"        健康服务器: {healthy_servers}")
    
    return "负载均衡和故障处理演示完成"

# 运行负载均衡演示
load_balancing_result = await demonstrate_load_balancing()
print(f"   {load_balancing_result}")

print(f"\n✅ 异步网络性能优化完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步网络性能分析和优化技术")
print(f"   ✓ 理解连接池、缓存和负载均衡机制")
print(f"   ✓ 学会网络监控和故障处理策略")
print(f"   ✓ 能够实现高效的异步网络系统")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**6.4 异步网络编程 [⭐⭐进阶]**
- ✅ 掌握异步HTTP客户端和服务器编程技术
- ✅ 学会使用aiohttp进行高效的异步网络请求
- ✅ 理解并发HTTP请求和性能优化
- ✅ 掌握高级HTTP客户端配置和重试机制
- ✅ 掌握异步WebSocket客户端和服务器技术
- ✅ 理解WebSocket连接管理和消息处理
- ✅ 学会实时数据流和双向通信实现
- ✅ 能够构建高效的WebSocket应用系统
- ✅ 掌握异步网络性能分析和优化技术
- ✅ 理解连接池、缓存和负载均衡机制
- ✅ 学会网络监控和故障处理策略
- ✅ 能够实现高效的异步网络系统
- ✅ 能独立优化异步网络性能

### 🎯 与LangChain学习的关联

**异步网络编程重要性**：
- 异步网络请求优化LangChain的API调用性能
- WebSocket支持LangChain的实时流式响应
- 并发HTTP请求提升LangChain的数据获取效率
- 异步网络编程增强LangChain的分布式能力
- 负载均衡和故障处理提升LangChain的可靠性

**实际应用场景**：
- LangChain的异步API调用和外部服务集成
- LangChain的实时流式响应和WebSocket通信
- LangChain的分布式模型推理和负载均衡
- LangChain的高并发数据处理和性能优化
- LangChain的网络监控和故障恢复机制

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习异步网络协议和自定义协议实现
   - 掌握异步网络安全和加密通信技术
   - 学习异步微服务架构和服务发现

2. **扩展学习**：
   - 了解异步网络编程的底层原理
   - 学习异步网络性能调优和监控工具
   - 探索分布式异步网络系统设计

3. **实际应用**：
   - 构建高性能异步API网关
   - 开发实时异步通信系统
   - 实现分布式异步任务调度系统

### 🔧 常见错误与注意事项

1. **异步HTTP客户端错误**：
   ```python
   # 错误：在异步函数中使用同步requests库
   async def bad_http_request():
       import requests  # 同步库，会阻塞事件循环
       response = requests.get('https://api.example.com')
       return response.json()
   
   # 正确：使用异步HTTP客户端
   async def good_http_request():
       async with aiohttp.ClientSession() as session:
           async with session.get('https://api.example.com') as response:
               return await response.json()
   ```

2. **WebSocket连接管理不当**：
   ```python
   # 错误：没有正确关闭WebSocket连接
   async def bad_websocket_usage():
       session = aiohttp.ClientSession()
       websocket = await session.ws_connect('wss://echo.example.com')
       await websocket.send_str('Hello')
       # 忘记关闭连接，导致资源泄漏
   
   # 正确：使用上下文管理器确保连接关闭
   async def good_websocket_usage():
       async with aiohttp.ClientSession() as session:
           async with session.ws_connect('wss://echo.example.com') as websocket:
               await websocket.send_str('Hello')
               response = await websocket.receive_str()
               return response
   ```

3. **并发控制不当**：
   ```python
   # 错误：无限制并发可能导致服务器拒绝服务
   async def bad_concurrent_requests(urls):
       async with aiohttp.ClientSession() as session:
           tasks = [session.get(url) for url in urls]  # 可能同时发起数千个请求
           return await asyncio.gather(*tasks)
   
   # 正确：使用信号量控制并发数
   async def good_concurrent_requests(urls, max_concurrent=10):
       semaphore = asyncio.Semaphore(max_concurrent)
       
       async def limited_request(session, url):
           async with semaphore:
               async with session.get(url) as response:
                   return await response.json()
       
       async with aiohttp.ClientSession() as session:
           tasks = [limited_request(session, url) for url in urls]
           return await asyncio.gather(*tasks)
   ```

4. **异常处理不完整**：
   ```python
   # 错误：没有处理网络异常
   async def bad_network_request():
       async with aiohttp.ClientSession() as session:
           async with session.get('https://api.example.com') as response:
               return await response.json()  # 如果网络中断会抛出异常
   
   # 正确：完整的异常处理
   async def good_network_request():
       try:
           async with aiohttp.ClientSession() as session:
               async with session.get('https://api.example.com') as response:
                   if response.status == 200:
                       return await response.json()
                   else:
                       raise aiohttp.ClientResponseError(
                           request_info=response.request_info,
                           history=response.history,
                           status=response.status
                       )
       except aiohttp.ClientConnectorError:
           print("连接失败")
           return None
       except asyncio.TimeoutError:
           print("请求超时")
           return None
       except Exception as e:
           print(f"未知错误: {e}")
           return None
   ```

5. **连接池配置不当**：
   ```python
   # 错误：连接池配置过小影响性能
   async def bad_connection_pool():
       connector = aiohttp.TCPConnector(limit=1)  # 只允许1个连接
       async with aiohttp.ClientSession(connector=connector) as session:
           # 所有请求都会串行执行，无法发挥并发优势
           pass
   
   # 正确：合理的连接池配置
   async def good_connection_pool():
       connector = aiohttp.TCPConnector(
           limit=100,        # 总连接数
           limit_per_host=20, # 每个主机的连接数
           ttl_dns_cache=300, # DNS缓存时间
           keepalive_timeout=30 # 保持连接时间
       )
       timeout = aiohttp.ClientTimeout(total=30, connect=5)
       async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
           # 优化的连接配置
           pass
   ```

6. **内存泄漏风险**：
   ```python
   # 错误：长时间运行的WebSocket没有心跳机制
   async def bad_websocket_long_running():
       async with aiohttp.ClientSession() as session:
           async with session.ws_connect('wss://stream.example.com') as websocket:
               while True:
                   message = await websocket.receive_str()  # 可能永远阻塞
                   process_message(message)
   
   # 正确：添加超时和心跳机制
   async def good_websocket_long_running():
       async with aiohttp.ClientSession() as session:
           async with session.ws_connect(
               'wss://stream.example.com',
               heartbeat=30  # 30秒心跳
           ) as websocket:
               while True:
                   try:
                       message = await asyncio.wait_for(
                           websocket.receive_str(), 
                           timeout=60.0
                       )
                       process_message(message)
                   except asyncio.TimeoutError:
                       # 发送心跳或处理超时
                       await websocket.ping()
   ```

### 🌐 性能优化建议

**HTTP客户端优化**：
- 使用连接池复用TCP连接
- 设置合理的超时和重试策略
- 启用HTTP/2和压缩支持
- 实现请求缓存和去重机制

**WebSocket优化**：
- 实现连接池和自动重连
- 使用心跳机制保持连接活跃
- 优化消息序列化和传输
- 实现消息缓冲和批量处理

**网络监控优化**：
- 实现实时性能监控和告警
- 使用分布式追踪和日志聚合
- 实现自适应负载均衡
- 监控网络资源使用情况

---

**🎉 恭喜完成异步网络编程学习！**

你已经全面掌握了异步网络编程的核心技术，能够系统性地进行高性能HTTP客户端开发、WebSocket实时通信和网络性能优化，为LangChain智能应用提供了强大的网络通信基础。

## 🚀 下一步学习预告

**第六节：异步编程 - 进行中** 🔄
- 6.1 异步编程基础 ✅
- 6.2 协程与事件循环 ✅
- 6.3 异步I/O操作 ✅
- 6.4 异步网络编程 ✅
- 6.5 异步Web框架 ⏳
- 6.6 并发编程模式 ⏳
- 6.7 异步任务队列 ⏳
- 6.8 异步性能优化 ⏳
- 6.9 异步最佳实践 ⏳

**继续第六节：异步编程**
- 下一个：6.5 异步Web框架
- 深入学习FastAPI和异步Web开发
- 掌握异步路由、中间件和依赖注入

**后续章节预告**：
- Web开发技术
- 项目工程实践

继续加油，异步网络编程已经掌握！准备深入学习异步Web框架！